In [1]:
import pandas as pd
import sqlite3
from ast import literal_eval

In [2]:
df = pd.read_excel("./2022-05-11.xlsx", sheet_name="Rede Prestadores", header=0)

In [3]:
places = pd.read_csv("./places.csv", header=0, dtype={ "contract": "string", "details": object })
places["details"] = places.details.apply(literal_eval)

In [4]:
df = df.drop_duplicates()

In [5]:
print(df.shape)
df.nunique()

(35400, 26)


CONTRATO                    5218
TIPO_PRESTADOR                 2
CNPJ_CPF                    4965
PRESTADOR                   4987
ATIVO                          1
REDE                           1
REDE_PADRAO                    2
NOME_FANTASIA_PONTO        10550
CEP                         5036
LOGRADOURO                  5100
NUMERO                      1989
COMPLEMENTO                 1395
BAIRRO                      1323
BAIRRO_AGRUPADOR               0
CIDADE                       373
UF                            27
DDD1                          64
FONE1                       7349
RAMAL1                        18
DDD2                          61
FONE2                       3479
RAMAL2                        13
CATEGORIA                      9
ESPECIALIDADE                173
ESPECIALIDADE_PRINCIPAL      123
TIPO_PRESTADOR.1               7
dtype: int64

In [6]:
df.head()

,CONTRATO,TIPO_PRESTADOR,CNPJ_CPF,PRESTADOR,ATIVO,REDE,REDE_PADRAO,NOME_FANTASIA_PONTO,CEP,LOGRADOURO,...,DDD1,FONE1,RAMAL1,DDD2,FONE2,RAMAL2,CATEGORIA,ESPECIALIDADE,ESPECIALIDADE_PRINCIPAL,TIPO_PRESTADOR.1
0,20113303,PJ,176910000000000000,2GN DIAGNOSTICOS POR IMAGEM LTDA,ATIVO,MASTER OURO,NAO,Q.IMED,27910210,R VELHO CAMPOS,...,22,27622410,NaN,22.0,27732811,NaN,SERVIÇO DE DIAGNÓSTICO,ULTRASSONOGRAFIA,ULTRASSONOGRAFIA,DIAGNÓSTICOS
1,20102780,PJ,3907080000000000000,A A STUDIO DE PATOLOGIA LTDA,ATIVO,MASTER OURO,NAO,A A STUDIO DE PATOLOGIA,20031202,R SENADOR DANTAS,...,21,22206849,NaN,21.0,22209397,NaN,SERVIÇO DE DIAGNÓSTICO,LABORATORIO / ANATOMIA PATOLOGICA,LABORATORIO / ANATOMIA PATOLOGICA,DIAGNÓSTICOS
2,20085036,PJ,6935800000000000,A ANDRE G POUSO LTDA,ATIVO,MASTER OURO,NAO,LABORATORIO PLANALTO DE PATOLOGIA CLINICA E CI...,72015525,AL CSB 2 ALAMEDA TORRE B,...,61,32451143,NaN,NaN,,NaN,SERVIÇO DE DIAGNÓSTICO,MEDICINA NUCLEAR,LABORATORIO / ANATOMIA PATOLOGICA,DIAGNÓSTICOS
3,20085036,PJ,6935800000000000,A ANDRE G POUSO LTDA,ATIVO,MASTER OURO,NAO,LABORATORIO PLANALTO DE PATOLOGIA CLINICA E CI...,72015525,AL CSB 2 ALAMEDA TORRE B,...,61,32451143,NaN,NaN,,NaN,SERVIÇO DE DIAGNÓSTICO,LABORATORIO / ANATOMIA PATOLOGICA,LABORATORIO / ANATOMIA PATOLOGICA,DIAGNÓSTICOS
4,20085036,PJ,6935800000000000,A ANDRE G POUSO LTDA,ATIVO,MASTER OURO,NAO,LABORATORIO PLANALTO DE PATOLOGIA CLINICA E CI...,72220120,STR SCS QD. 1 BLOCO B SL 1004,...,61,33515115,NaN,61.0,32451143,NaN,SERVIÇO DE DIAGNÓSTICO,MEDICINA NUCLEAR,LABORATORIO / ANATOMIA PATOLOGICA,DIAGNÓSTICOS


In [491]:
import googlemaps
gmaps = googlemaps.Client(key="--redacted--")

In [518]:
from collections import defaultdict

def build_addr(x):
    addr = ""
    if x.NOME_FANTASIA_PONTO:
        addr += f"{x.NOME_FANTASIA_PONTO} - "
#     if x.LOGRADOURO:
#         addr += f"{x.LOGRADOURO} "
#     if x.NUMERO:
#         addr += f"{x.NUMERO}, "
    if x.BAIRRO:
        addr += f"{x.BAIRRO}, "
    if x.CIDADE:
        addr += f"{x.CIDADE}, "
    if x.UF:
        addr += f"{x.UF}, "
    addr += "Brasil"

    return addr


def build_phone(x):
    return f"+55 {x.DDD1} {x.FONE1}"

place_ids = set()
failed_results = defaultdict(list)

def get_place_data(row):
#     r = gmaps.find_place(build_addr(row), "textquery")
    r = gmaps.find_place(build_phone(row), "phonenumber")
    if r["status"] != "OK":
        print("find place failed", row.CONTRATO)
        return None
    for candidate in r["candidates"]:
        if candidate["place_id"] in place_ids:
            continue
        place_ids.add(candidate["place_id"])
        d = gmaps.place(candidate["place_id"], language="pt-BR")
        if d["status"] != "OK":
            print("details failed", row.CONTRATO)
            continue
        result = d["result"]
        if not ("health" in result["types"] or "doctor" in result["types"]):
            print("incorrect type")
            failed_results[row.CONTRATO].append(result)
            continue
        return result


def find_group_place_data(group):
    for _, row in group.iterrows():
        data = get_place_data(row)
        if data is not None:
            return data
    
    
def save_places():
    places.to_csv("./places.csv", index=False)

def append_place(contract, details):
    places.loc[len(places)] = [contract, details]


In [519]:
idx = 0
for contract, group in df.groupby("CONTRATO"):
    # Skip existing
    if str(contract) in places.contract.values:
        continue
    data = find_group_place_data(group)
    if data:
        append_place(contract, data)

    if idx % 10 == 0:
        save_places()
    idx += 1

save_places()

find place failed 10073990
incorrect type
find place failed 10089543
find place failed 10089543
find place failed 10099824
find place failed 12000230
find place failed 19303525
find place failed 19303525
find place failed 19306513
find place failed 19306970
find place failed 19307230
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
find place failed 20069723
find place failed 20071418
find place failed 20089090
find place failed 20089430
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
incorrect type
find place failed 20112521
find place failed 20112521
find place failed 20112521
find place failed 20112521
find place failed 20112521
find place failed 20112521
find place failed 20112521
find place failed 20112521
incorrect type
incorrect type
incorrect type
i

find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680674
find place failed 30680675
find place failed 30680675
find place failed 30680875
find place failed 30680974
find place failed 30681002
find place failed 30681023
find place failed 30681023
find place failed 30681025
find place failed 30681025
find place failed 30681029
find place failed 30681049
find place failed 30681049
find place failed 30681049
f

In [7]:
mdf = df.copy()
mdf["contract"] = mdf["CONTRATO"].apply(str).astype('string')

In [8]:
places["contract"] = places["contract"].apply(str).astype('string')

In [9]:
joined = mdf.join(places.set_index("contract"), "contract").dropna(subset=["details"]).drop_duplicates("contract").sort_values("contract").reset_index().drop("index", axis=1)

In [16]:
provider = joined[["contract", "REDE", "TIPO_PRESTADOR.1"]].rename(columns={"REDE": "network", "TIPO_PRESTADOR.1": "type"})
provider["network"] = provider["network"].str.title()
provider["type"] = provider["type"].str.title()
provider["rating"] = joined["details"].apply(lambda x: x.get("rating"))
provider["total_ratings"] = joined["details"].apply(lambda x: x.get("user_ratings_total", 0))
provider["website"] = joined["details"].apply(lambda x: x.get("website"))
provider["google_url"] = joined["details"].apply(lambda x: x["url"])
provider["name"] = joined["details"].apply(lambda x: x["name"])
provider["phone_number"] = joined["details"].apply(lambda x: x.get("formatted_phone_number"))
provider["status"] = joined["details"].apply(lambda x: x["business_status"])

In [17]:
provider.to_csv("./provider.csv", index=False)

In [18]:
details = joined[["contract", "details"]]

In [12]:
location = details[["contract"]]
location["lat"] = details["details"].apply(lambda x: x["geometry"]["location"]["lat"])
location["lng"] = details["details"].apply(lambda x: x["geometry"]["location"]["lng"])
location["address"] = details["details"].apply(lambda x: x["formatted_address"])

def find_component(target, long):
    def find_sub(r):
        for component in r["address_components"]:
            if target in component["types"]:
                return component["long_name" if long else "short_name"]
    return find_sub

location["postal_code"] = details["details"].apply(find_component("postal_code", True))
location["country"] = details["details"].apply(find_component("country", True))
location["state"] = details["details"].apply(find_component("administrative_area_level_1", False))
location["city"] = details["details"].apply(find_component("administrative_area_level_2", True))
location = location.set_index("contract")

In [ ]:
location.to_csv("./location.csv")

In [13]:
reviews = details[["contract"]]
reviews["review"] = details["details"].apply(lambda x: x.get("reviews", []))
reviews = reviews.explode("review").dropna(subset=["review"]).reset_index(drop=True)
reviews["author_name"] = reviews["review"].apply(lambda x: x["author_name"])
reviews["author_photo_url"] = reviews["review"].apply(lambda x: x.get("profile_photo_url"))
reviews["rating"] = reviews["review"].apply(lambda x: x["rating"])
reviews["text"] = reviews["review"].apply(lambda x: x["text"])
reviews["time"] = reviews["review"].apply(lambda x: x["time"])
reviews = reviews.set_index("contract").drop("review", axis=1)

In [ ]:
reviews.to_csv("./review.csv")

In [14]:
esp = df[["CONTRATO", "ESPECIALIDADE", "ESPECIALIDADE_PRINCIPAL"]]
espc = esp.copy()
espc["ESPECIALIDADE"] = espc["ESPECIALIDADE_PRINCIPAL"]
esp = pd.concat([esp, espc]).drop_duplicates().sort_values(["CONTRATO", "ESPECIALIDADE"])
esp["ESPECIALIDADE"] = esp["ESPECIALIDADE"].str.title()
esp["ESPECIALIDADE_PRINCIPAL"] = esp["ESPECIALIDADE_PRINCIPAL"].str.title()
esp["ESPECIALIDADE_PRINCIPAL"] = esp.apply(lambda r: r["ESPECIALIDADE"] == r["ESPECIALIDADE_PRINCIPAL"], axis=1)
esp = esp.rename(columns={"CONTRATO": "contract", "ESPECIALIDADE": "specialty", "ESPECIALIDADE_PRINCIPAL": "is_primary"}).set_index("contract")

In [ ]:
esp.to_csv("./especialidades.csv")

In [15]:
cat = df[["CONTRATO", "CATEGORIA"]].drop_duplicates().sort_values(["CONTRATO", "CATEGORIA"])
cat["CATEGORIA"] = cat["CATEGORIA"].str.title()
cat = cat.rename(columns={"CONTRATO": "contract", "CATEGORIA": "category"}).set_index("contract")

In [ ]:
cat.to_csv("./categorias.csv")

In [19]:
import sqlite3 as sql

con = sql.connect("./db.sqlite")

con.execute("DROP TABLE IF EXISTS provider;")
con.execute("""
CREATE TABLE "provider" (
  "contract" text PRIMARY KEY NOT NULL,
  "network" text NOT NULL,
  "type" text NOT NULL,
  "rating" real,
  "total_ratings" int NOT NULL,
  "website" text,
  "google_url" text NOT NULL,
  "name" text NOT NULL,
  "phone_number" text,
  "status" text NOT NULL
);
""")
provider.set_index("contract").to_sql("provider", con=con, if_exists="append")

con.execute("DROP TABLE IF EXISTS location;")
con.execute("""
CREATE TABLE "location" (
  "id" integer PRIMARY KEY AUTOINCREMENT NOT NULL,
  "contract" text NOT NULL,
  "lat" real NOT NULL,
  "lng" real NOT NULL,
  "address" text NOT NULL,
  "postal_code" text,
  "country" text NOT NULL,
  "state" text NOT NULL,
  "city" text NOT NULL,
  FOREIGN KEY ("contract") REFERENCES "main" ("contract")
);
""")
location.to_sql("location", con=con, if_exists="append")

con.execute("DROP TABLE IF EXISTS review;")
con.execute("""
CREATE TABLE "review" (
  "id" integer PRIMARY KEY AUTOINCREMENT NOT NULL,
  "contract" text NOT NULL,
  "author_name" text NOT NULL,
  "author_photo_url" text,
  "rating" int NOT NULL,
  "text" text NOT NULL,
  "time" timestamp NOT NULL,
  FOREIGN KEY ("contract") REFERENCES "main" ("contract")
);
""")
reviews.to_sql("review", con=con, if_exists="append")

con.execute("DROP TABLE IF EXISTS category;")
con.execute("""
CREATE TABLE "category" (
  "id" integer PRIMARY KEY AUTOINCREMENT NOT NULL,
  "contract" text NOT NULL,
  "category" text NOT NULL,
  FOREIGN KEY ("contract") REFERENCES "main" ("contract")
);
""")
cat.to_sql("category", con=con, if_exists="append")

con.execute("DROP TABLE IF EXISTS specialty;")
con.execute("""
CREATE TABLE "specialty" (
  "id" integer PRIMARY KEY AUTOINCREMENT NOT NULL,
  "contract" text NOT NULL,
  "specialty" text NOT NULL,
  "is_primary" int NOT NULL,
  FOREIGN KEY ("contract") REFERENCES "main" ("contract")
);
""")
esp.to_sql("specialty", con=con, if_exists="append")

24119

In [971]:
con.close()